In [11]:
!pip install plyer


import dash
import dash_table
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State, ALL
import sqlite3
from datetime import datetime
import pandas as pd
from threading import Thread
from IPython.display import display, HTML

#########################
# Dashboard Layout / View
#########################

app = dash.Dash(__name__)
app.layout = html.Div([

    # Title
    html.Center(html.B(html.H1("Event Tracker"))),

    # Login page
    html.Div(id='login-div',
             children=[
                 html.H2("Login/Register"),
                 dcc.Input(id='username', type='text', placeholder='Username'),
                 dcc.Input(id='password', type='password', placeholder='Password'),
                 html.Button('Login/Register', id='login-button'),
                 html.Div(id='login-output')
             ]
    ),

    # Events page
    html.Div(id='event-div',
             style={'display': 'none'},
             children=[
                 # Display section to add events
                 html.H2("Add Event"),
                 dcc.Input(id='event-name', type='text', placeholder='Event Name'),
                 dcc.Input(id='event-date', type='text', placeholder='Event Date (YYYY-MM-DD)'),
                 dcc.Input(id='event-time', type='text', placeholder='Event Time (HH:MM)'),
                 dcc.Input(id='event-location', type='text', placeholder='Event Location'),
                 dcc.Input(id='event-description', type='text', placeholder='Event Description'),
                 html.Button('Add Event', id='add-event-button'),
                 html.Div(id='add-event-output'),

                 # Display section of upcoming events with a table
                 html.H2("Upcoming Events"),
                 dash_table.DataTable(id='events-table', columns=[
                     {"name": "Event Name", "id": "event_name"},
                     {"name": "Date", "id": "event_date"},
                     {"name": "Time", "id": "event_time"},
                     {"name": "Location", "id": "event_location"},
                     {"name": "Description", "id": "event_description"},
                     {"name": "Edit", "id": "edit_event", "presentation": "markdown"},
                     {"name": "Delete", "id": "delete_event", "presentation": "markdown"}
                 ],
                 editable=True,
                 row_deletable=True)
             ]
    )
])

# Database Functions
def get_events(user_id):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute("SELECT * FROM events WHERE user_id = ?", (user_id,))
    events = c.fetchall()
    conn.close()
    return events

def add_event(user_id, name, date, time, location, description):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute("INSERT INTO events (user_id, event_name, event_date, event_time, event_location, event_description) VALUES (?, ?, ?, ?, ?, ?)",
              (user_id, name, date, time, location, description))
    conn.commit()
    conn.close()

def edit_event(event_id, name, date, time, location, description):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute("UPDATE events SET event_name = ?, event_date = ?, event_time = ?, event_location = ?, event_description = ? WHERE id = ?",
              (name, date, time, location, description, event_id))
    conn.commit()
    conn.close()

def remove_event(event_id):
    conn = sqlite3.connect('events.db')
    c = conn.cursor()
    c.execute("DELETE FROM events WHERE id = ?", (event_id,))
    conn.commit()
    conn.close()

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('login-output', 'children'),
     Output('login-div', 'style'),
     Output('event-div', 'style'),
     Output('events-table', 'data'),
     Output('add-event-output', 'children')],
    [Input('login-button', 'n_clicks'),
     Input('add-event-button', 'n_clicks'),
     Input({'type': 'edit-event', 'index': ALL}, 'n_clicks'),
     Input({'type': 'delete-event', 'index': ALL}, 'n_clicks')],
    [State('username', 'value'),
     State('password', 'value'),
     State('event-name', 'value'),
     State('event-date', 'value'),
     State('event-time', 'value'),
     State('event-location', 'value'),
     State('event-description', 'value'),
     State('events-table', 'data')],
    prevent_initial_call=True
)
def handle_callbacks(login_clicks, add_event_clicks, edit_clicks, delete_clicks, username, password, name, date, time, location, description, events_data):
    ctx = dash.callback_context

    if not ctx.triggered:
        return '', {'display': 'block'}, {'display': 'none'}, [], ''

    trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]

    # If the login button was clicked
    if trigger_id == 'login-button':
        conn = sqlite3.connect('events.db')
        c = conn.cursor()
        c.execute("SELECT * FROM users WHERE username = ?", (username,))
        user = c.fetchone()

        if user and user[2] == password:
            user_id = user[0]
            events = get_events(user_id)
            event_data = [{'id': e[0], 'event_name': e[2], 'event_date': e[3], 'event_time': e[4], 'event_location': e[5], 'event_description': e[6],
                           'edit_event': f'Edit', 'delete_event': f'Delete'} for e in events]
            return f'Logged in as {username}', {'display': 'none'}, {'display': 'block'}, event_data, ''

        elif not user:
            c.execute("INSERT INTO users (username, password) VALUES (?, ?)", (username, password))
            conn.commit()
            user_id = c.lastrowid
            conn.close()
            return f'Logged in as {username}', {'display': 'none'}, {'display': 'block'}, [], ''

        else:
            return 'Incorrect username or password', {'display': 'block'}, {'display': 'none'}, [], ''

    # Add event button
    elif trigger_id == 'add-event-button':
        conn = sqlite3.connect('events.db')
        c = conn.cursor()
        c.execute("SELECT id FROM users WHERE username = ?", (username,))
        user_id = c.fetchone()[0]
        conn.close()

        add_event(user_id, name, date, time, location, description)
        events = get_events(user_id)
        event_data = [{'id': e[0], 'event_name': e[2], 'event_date': e[3], 'event_time': e[4], 'event_location': e[5], 'event_description': e[6],
                       'edit_event': f'Edit', 'delete_event': f'Delete'} for e in events]
        return '', {'display': 'none'}, {'display': 'block'}, event_data, 'Event added successfully'

    elif 'edit-event' in trigger_id:
        edit_index = int(trigger_id.split('.')[1])
        event_id = events_data[edit_index]['id']
        edit_event(event_id, name, date, time, location, description)
        events = get_events(user_id)
        event_data = [{'id': e[0], 'event_name': e[2], 'event_date': e[3], 'event_time': e[4], 'event_location': e[5], 'event_description': e[6],
                       'edit_event': f'Edit', 'delete_event': f'Delete'} for e in events]
        return '', {'display': 'none'}, {'display': 'block'}, event_data, 'Event edited successfully'

    elif 'delete-event' in trigger_id:
        delete_index = int(trigger_id.split('.')[1])
        event_id = events_data[delete_index]['id']
        remove_event(event_id)
        events = get_events(user_id)
        event_data = [{'id': e[0], 'event_name': e[2], 'event_date': e[3], 'event_time': e[4], 'event_location': e[5], 'event_description': e[6],
                       'edit_event': f'Edit', 'delete_event': f'Delete'} for e in events]
        return '', {'display': 'none'}, {'display': 'block'}, event_data, 'Event deleted successfully'

    return '', {'display': 'block'}, {'display': 'none'}, [], ''

# Function to run the server in a separate thread
def run_server():
    app.run_server(debug=True, use_reloader=False)

# Start the server
thread = Thread(target=run_server)
thread.start()

# Display the link in Jupyter Notebook
display(HTML(f'<a href="http://localhost:8050" target="_blank">Click here to open the Dash app</a>'))


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
